## Download Radolan .bin.gz files from DWD ftp server

To-Do
Ordner im System erstellen mit der jeweiligen Jahreszahl der Daten klappts?
Nicht die Dateien Downloaden die bereits lokal im Ordner sind 
        '''Create directory structure if it doesn't exist'''
    if not os.path.exists(os.path.dirname(key)):
        os.makedirs(os.path.dirname(key))
Entpacken?

## Version 1 (historical) without tarfile checking only if file is existing on disk (better v2)

In [1]:
#!pip install pyftpdlib
import ftplib
import os, os.path
from pathlib import Path
import tarfile #Check if tarfile is corrupted, important when script is run twice 
BLOCK_SIZE = 1024

#working dir
windows = "C:/Users/Elisabeth/Desktop"
ubuntu = "/home/elisabeth/Dokumente/master/radolan_data/raw"
download_directory = "radolan_downloads" #Directory where files will be saved
os.chdir(windows) 
if not os.path.exists(download_directory):
    os.makedirs(download_directory)
os.chdir(download_directory)
print("Actual Working dir: ", os.getcwd())

#location of the file in the ftp server
path = 'climate_environment/CDC/grids_germany/daily/radolan/historical/bin/'
host = "https://opendata.dwd.de/"
ip =  "141.38.2.26" 

#Create Connection
ftp = ftplib.FTP(ip) 

#Login
ftp.login()

#Change Path to Radolan bin directory
ftp.cwd(path)

#Get list of Years available 
paths = ftp.nlst()
print("Available Years: ", paths)

#Create local directories
for path in paths:
    if not os.path.exists(path):
        os.makedirs(path)

#Workaround for problem that if i cahnged first time i am not in right directory
count = True
#For loop to download all files from every year directory 
for path in paths:
    if count == True:
        count = False 
        ftp.cwd(path)
        print("Path: ", ftp.pwd())
        filenames = ftp.nlst()
        print("Files stored in this path: ", filenames)
        os.chdir(path)
        for file in filenames:
            print("Start downloading file: ", file)
            if os.path.isfile(file) and os.access(file, os.R_OK)== True:
                print(file, "already exists!")
                continue
            else:
                handle = open(file.lstrip("/"), 'wb') 
                ftp.retrbinary('RETR %s' % file, handle.write)
                print("File " + file + " downloaded succsefully!")
                continue
        os.chdir('../')
    else:
        ftp.cwd(("../"))
        ftp.cwd(path)
        count = False 
        print("Path: ", ftp.pwd())
        print("Working dir : ", os.getcwd())
        filenames = ftp.nlst()
        print("Files stored in this path: ", filenames)
        os.chdir(path)
        for file in filenames:
            print("Start downloading file: ", file)
            if os.path.isfile(file) and os.access(file, os.R_OK) == True:
                print(file, "already exists!")
                continue
            else:
                handle = open(file.lstrip("/"), 'wb') 
                ftp.retrbinary('RETR %s' % file, handle.write)
                print("File " + file + " downloaded succsefully!")
                continue
        os.chdir('../')

                    
print("Job done... connection closed")
#close connection
ftp.quit()


Actual Working dir:  C:\Users\Elisabeth\Desktop\radolan_downloads
Available Years:  ['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017']
Path:  /climate_environment/CDC/grids_germany/daily/radolan/historical/bin/2006
Files stored in this path:  ['SF-200610.tar.gz', 'SF-200611.tar.gz', 'SF-200612.tar.gz']
Start downloading file:  SF-200610.tar.gz
SF-200610.tar.gz already exists!
Start downloading file:  SF-200611.tar.gz
SF-200611.tar.gz already exists!
Start downloading file:  SF-200612.tar.gz
SF-200612.tar.gz already exists!
Path:  /climate_environment/CDC/grids_germany/daily/radolan/historical/bin/2007
Working dir :  C:\Users\Elisabeth\Desktop\radolan_downloads
Files stored in this path:  ['SF-200701.tar.gz', 'SF-200702.tar.gz', 'SF-200703.tar.gz', 'SF-200704.tar.gz', 'SF-200705.tar.gz', 'SF-200706.tar.gz', 'SF-200707.tar.gz', 'SF-200708.tar.gz', 'SF-200709.tar.gz', 'SF-200710.tar.gz', 'SF-200711.tar.gz', 'SF-200712.tar.gz']
Start downloadin

File SF201206.tar.gz downloaded succsefully!
Start downloading file:  SF201207.tar.gz
File SF201207.tar.gz downloaded succsefully!
Start downloading file:  SF201208.tar.gz
File SF201208.tar.gz downloaded succsefully!
Start downloading file:  SF201209.tar.gz
File SF201209.tar.gz downloaded succsefully!
Start downloading file:  SF201210.tar.gz
File SF201210.tar.gz downloaded succsefully!
Start downloading file:  SF201211.tar.gz
File SF201211.tar.gz downloaded succsefully!
Start downloading file:  SF201212.tar.gz
File SF201212.tar.gz downloaded succsefully!
Path:  /climate_environment/CDC/grids_germany/daily/radolan/historical/bin/2013
Working dir :  C:\Users\Elisabeth\Desktop\radolan_downloads
Files stored in this path:  ['SF201301.tar.gz', 'SF201302.tar.gz', 'SF201303.tar.gz', 'SF201304.tar.gz', 'SF201305.tar.gz', 'SF201306.tar.gz', 'SF201307.tar.gz', 'SF201308.tar.gz', 'SF201309.tar.gz', 'SF201310.tar.gz', 'SF201311.tar.gz', 'SF201312.tar.gz']
Start downloading file:  SF201301.tar.gz
F

'221 Goodbye.'

## Version 2 (historical) with tar file checking of integrety?!

In [4]:
#!pip install pyftpdlib
import ftplib
import os, os.path
from pathlib import Path
import tarfile #Check if tarfile is corrupted, important when script is run twice 
BLOCK_SIZE = 1024

#working dir
windows = "C:/Users/Elisabeth/Desktop"
ubuntu = "/home/elisabeth/Dokumente/master/radolan_data/raw"
download_directory = "radolan_downloads" #Directory where files will be saved
os.chdir(windows) 
if not os.path.exists(download_directory):
    os.makedirs(download_directory)
os.chdir(download_directory)
print("Actual Working dir: ", os.getcwd())

#location of the file in the ftp server
#historical dir: 
path = 'climate_environment/CDC/grids_germany/daily/radolan/historical/bin/'
#recent dir:
path = 
host = "https://opendata.dwd.de/"
ip =  "141.38.2.26" 

#Create Connection
ftp = ftplib.FTP(ip) 

#Login
ftp.login()

#Change Path to Radolan bin directory
ftp.cwd(path)

#Get list of Years available 
paths = ftp.nlst()
print("Available Years: ", paths)

#Create local directories
for path in paths:
    if not os.path.exists(path):
        os.makedirs(path)
   

#Workaround for problem that if i cahnged first time i am not in right directory
count = True
#For loop to download all files from every year directory 
for path in paths:
    if count == True:
        count = False 
        ftp.cwd(path) #Change path to actuall path from paths list
        print("Path: ", ftp.pwd())
        filenames = ftp.nlst() #get files from directory
        print("Files stored in this path: ", filenames)
        os.chdir(path) # change dir from local drive
        for file in filenames: #loop through all files inside driectory to download them
            print("Check if file is available on disk: ", file) #try to open existing tar files and check if they are valid
            try: 
                tardude = tarfile.open(file) #tarfile object
            except:
                print( "There was an error opening tarfile. The file might be corrupt or missing")
            try:
                members = tardude.getmembers() #tarinfo object
            except:
                print("Error reading tarfile membrs")
            for member_info in members:
                try:
                    check = tardude.extractfile(member_info.name)
                    while 1:
                        data = check.read(BLOCK_SIZE)
                        if not data:
                            break
                except: #if file is in any way not local or hsae any error download it
                    print("Tar file is corrupt, start downloading again...")
                    handle = open(file.lstrip("/"), 'wb') 
                    ftp.retrbinary('RETR %s' % file, handle.write)
                    print("File " + file + " downloaded succsefully!")
            ftp.sendcmd('PASV') # To avoid getting 401 timeout from ftp server
            print(file, "seems to be valid an available...")
        os.chdir('../') #go folder backwards
    else:
        ftp.cwd(("../"))
        ftp.cwd(path)
        count = False 
        print("Path: ", ftp.pwd())
        print("Working dir : ", os.getcwd())
        filenames = ftp.nlst()
        print("Files stored in this path: ", filenames)
        os.chdir(path)
        for file in filenames:
            print("Check if file is available on disk: ", file)
            try: 
                tardude = tarfile.open(file)
            except:
                print( "There was an error opening tarfile. The file might be corrupt or missing ")
            try:
                members = tardude.getmembers()
            except:
                print("Error reading tarfile membrs")
            for member_info in members:
                try:
                    check = tardude.extractfile(member_info.name)
                    while 1:
                        data = check.read(BLOCK_SIZE)
                        if not data:
                            break
                except:
                    print("Tar file is corrupt, start downloading again...")
                    handle = open(file.lstrip("/"), 'wb') 
                    ftp.retrbinary('RETR %s' % file, handle.write)
                    print("File " + file + " downloaded succsefully!")
                    break
            ftp.sendcmd('PASV') # To avoid getting 401 timeout from ftp server
            print(file, "seems to be valid an available...")
        os.chdir('../')

                    
print("Job done... connection closed")
#close connection
ftp.quit()


Actual Working dir:  C:\Users\Elisabeth\Desktop\radolan_downloads
Available Years:  ['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017']
Path:  /climate_environment/CDC/grids_germany/daily/radolan/historical/bin/2006
Files stored in this path:  ['SF-200610.tar.gz', 'SF-200611.tar.gz', 'SF-200612.tar.gz']
Check if file is available on disk:  SF-200610.tar.gz
SF-200610.tar.gz seems to be valid an available...
Check if file is available on disk:  SF-200611.tar.gz
SF-200611.tar.gz seems to be valid an available...
Check if file is available on disk:  SF-200612.tar.gz
SF-200612.tar.gz seems to be valid an available...
Path:  /climate_environment/CDC/grids_germany/daily/radolan/historical/bin/2007
Working dir :  C:\Users\Elisabeth\Desktop\radolan_downloads
Files stored in this path:  ['SF-200701.tar.gz', 'SF-200702.tar.gz', 'SF-200703.tar.gz', 'SF-200704.tar.gz', 'SF-200705.tar.gz', 'SF-200706.tar.gz', 'SF-200707.tar.gz', 'SF-200708.tar.gz', 'SF-20

SF201101.tar.gz seems to be valid an available...
Check if file is available on disk:  SF201102.tar.gz
SF201102.tar.gz seems to be valid an available...
Check if file is available on disk:  SF201103.tar.gz
SF201103.tar.gz seems to be valid an available...
Check if file is available on disk:  SF201104.tar.gz
SF201104.tar.gz seems to be valid an available...
Check if file is available on disk:  SF201105.tar.gz
SF201105.tar.gz seems to be valid an available...
Check if file is available on disk:  SF201106.tar.gz
SF201106.tar.gz seems to be valid an available...
Check if file is available on disk:  SF201107.tar.gz
SF201107.tar.gz seems to be valid an available...
Check if file is available on disk:  SF201108.tar.gz
SF201108.tar.gz seems to be valid an available...
Check if file is available on disk:  SF201109.tar.gz
SF201109.tar.gz seems to be valid an available...
Check if file is available on disk:  SF201110.tar.gz
SF201110.tar.gz seems to be valid an available...
Check if file is availa

SF201602.tar.gz seems to be valid an available...
Check if file is available on disk:  SF201603.tar.gz
SF201603.tar.gz seems to be valid an available...
Check if file is available on disk:  SF201604.tar.gz
SF201604.tar.gz seems to be valid an available...
Check if file is available on disk:  SF201605.tar.gz
SF201605.tar.gz seems to be valid an available...
Check if file is available on disk:  SF201606.tar.gz
SF201606.tar.gz seems to be valid an available...
Check if file is available on disk:  SF201607.tar.gz
SF201607.tar.gz seems to be valid an available...
Check if file is available on disk:  SF201608.tar.gz
SF201608.tar.gz seems to be valid an available...
Check if file is available on disk:  SF201609.tar.gz
SF201609.tar.gz seems to be valid an available...
Check if file is available on disk:  SF201610.tar.gz
SF201610.tar.gz seems to be valid an available...
Check if file is available on disk:  SF201611.tar.gz
SF201611.tar.gz seems to be valid an available...
Check if file is availa

'221 Goodbye.'

## Version 1 (recent data) download radolan data from ftp server 

In [5]:
#!pip install pyftpdlib
import ftplib
import os, os.path
from pathlib import Path
import tarfile #Check if tarfile is corrupted, important when script is run twice 
BLOCK_SIZE = 1024

#Time to be derived hhmm
time = "2350"

#working dir
windows = "C:/Users/Elisabeth/Desktop"
ubuntu = "/home/elisabeth/Dokumente/master/radolan_data/raw"
download_directory = "radolan_downloads" #Directory where files will be saved
os.chdir(windows) 

if not os.path.exists(download_directory):
    os.makedirs(download_directory)
os.chdir(download_directory)
print("Actual Working dir: ", os.getcwd())

#location of the file in the ftp server
#historical dir: 
#path = 'climate_environment/CDC/grids_germany/daily/radolan/historical/bin/'
#recent dir:
path = 'climate_environment/CDC/grids_germany/daily/radolan/recent/bin/' 
host = "https://opendata.dwd.de/"
ip =  "141.38.2.26" 

#Create Connection
ftp = ftplib.FTP(ip) 

#Login
ftp.login()

#Change Path to Radolan bin directory
ftp.cwd(path)

#Find out which years handle in recent
files = ftp.nlst() #list of all files in directory
files_f = files[2:] #exclude first two elemnts they are descriptions
print("Available files: ", files_f[0:25]) #First 25 files; example: raa01-sf_10000-1801010050-dwd---bin.gz
years = [x[15:17] for x in files_f] #cut string to years
years_f = list(dict.fromkeys(years))#delete duplicates with creating dict from list keys (this automatically delete duplicates) and then tolist
paths = []
for year in years_f: #Add 20 to 
    paths.append("20"+year)
    
#Create local directories
for path in paths:
    if not os.path.exists(path):
        os.makedirs(path)

os.chdir(paths[0])

#List comprehension - get only filenames with specific time
filenames = [x for x in files_f if time in x]

for file in filenames[0:365]:
    print("Start downloading: ", file)
    handle = open(file, "wb")
    ftp.retrbinary("RETR %s" %file, handle.write)
    print("Download of ",file, "succesfully!") 
   
os.chdir("../")
os.chdir(paths[1])

for file in filenames[365:]:
    print("Start downloading: ", file)
    handle = open(file, "wb")
    ftp.retrbinary("RETR %s" %file, handle.write)
    print("Download of ",file, "succesfully!") 
    
print("Job done... connection closed")
#close connection
ftp.quit()


Actual Working dir:  C:\Users\Elisabeth\Desktop\radolan_downloads
Available files:  ['raa01-sf_10000-1801010050-dwd---bin.gz', 'raa01-sf_10000-1801010150-dwd---bin.gz', 'raa01-sf_10000-1801010250-dwd---bin.gz', 'raa01-sf_10000-1801010350-dwd---bin.gz', 'raa01-sf_10000-1801010450-dwd---bin.gz', 'raa01-sf_10000-1801010550-dwd---bin.gz', 'raa01-sf_10000-1801010650-dwd---bin.gz', 'raa01-sf_10000-1801010750-dwd---bin.gz', 'raa01-sf_10000-1801010850-dwd---bin.gz', 'raa01-sf_10000-1801010950-dwd---bin.gz', 'raa01-sf_10000-1801011050-dwd---bin.gz', 'raa01-sf_10000-1801011150-dwd---bin.gz', 'raa01-sf_10000-1801011250-dwd---bin.gz', 'raa01-sf_10000-1801011350-dwd---bin.gz', 'raa01-sf_10000-1801011450-dwd---bin.gz', 'raa01-sf_10000-1801011550-dwd---bin.gz', 'raa01-sf_10000-1801011650-dwd---bin.gz', 'raa01-sf_10000-1801011750-dwd---bin.gz', 'raa01-sf_10000-1801011850-dwd---bin.gz', 'raa01-sf_10000-1801011950-dwd---bin.gz', 'raa01-sf_10000-1801012050-dwd---bin.gz', 'raa01-sf_10000-1801012150-dwd---

Download of  raa01-sf_10000-1802272350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1802282350-dwd---bin.gz
Download of  raa01-sf_10000-1802282350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1803012350-dwd---bin.gz
Download of  raa01-sf_10000-1803012350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1803022350-dwd---bin.gz
Download of  raa01-sf_10000-1803022350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1803032350-dwd---bin.gz
Download of  raa01-sf_10000-1803032350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1803042350-dwd---bin.gz
Download of  raa01-sf_10000-1803042350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1803052350-dwd---bin.gz
Download of  raa01-sf_10000-1803052350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1803062350-dwd---bin.gz
Download of  raa01-sf_10000-1803062350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1803072350-dwd---bin.gz
Download

Download of  raa01-sf_10000-1805052350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1805062350-dwd---bin.gz
Download of  raa01-sf_10000-1805062350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1805072350-dwd---bin.gz
Download of  raa01-sf_10000-1805072350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1805082350-dwd---bin.gz
Download of  raa01-sf_10000-1805082350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1805092350-dwd---bin.gz
Download of  raa01-sf_10000-1805092350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1805102350-dwd---bin.gz
Download of  raa01-sf_10000-1805102350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1805112350-dwd---bin.gz
Download of  raa01-sf_10000-1805112350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1805122350-dwd---bin.gz
Download of  raa01-sf_10000-1805122350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1805132350-dwd---bin.gz
Download

Download of  raa01-sf_10000-1807112350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1807122350-dwd---bin.gz
Download of  raa01-sf_10000-1807122350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1807132350-dwd---bin.gz
Download of  raa01-sf_10000-1807132350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1807142350-dwd---bin.gz
Download of  raa01-sf_10000-1807142350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1807152350-dwd---bin.gz
Download of  raa01-sf_10000-1807152350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1807162350-dwd---bin.gz
Download of  raa01-sf_10000-1807162350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1807172350-dwd---bin.gz
Download of  raa01-sf_10000-1807172350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1807182350-dwd---bin.gz
Download of  raa01-sf_10000-1807182350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1807192350-dwd---bin.gz
Download

Download of  raa01-sf_10000-1809162350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1809172350-dwd---bin.gz
Download of  raa01-sf_10000-1809172350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1809182350-dwd---bin.gz
Download of  raa01-sf_10000-1809182350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1809192350-dwd---bin.gz
Download of  raa01-sf_10000-1809192350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1809202350-dwd---bin.gz
Download of  raa01-sf_10000-1809202350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1809212350-dwd---bin.gz
Download of  raa01-sf_10000-1809212350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1809222350-dwd---bin.gz
Download of  raa01-sf_10000-1809222350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1809232350-dwd---bin.gz
Download of  raa01-sf_10000-1809232350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1809242350-dwd---bin.gz
Download

Download of  raa01-sf_10000-1811222350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1811232350-dwd---bin.gz
Download of  raa01-sf_10000-1811232350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1811242350-dwd---bin.gz
Download of  raa01-sf_10000-1811242350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1811252350-dwd---bin.gz
Download of  raa01-sf_10000-1811252350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1811262350-dwd---bin.gz
Download of  raa01-sf_10000-1811262350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1811272350-dwd---bin.gz
Download of  raa01-sf_10000-1811272350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1811282350-dwd---bin.gz
Download of  raa01-sf_10000-1811282350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1811292350-dwd---bin.gz
Download of  raa01-sf_10000-1811292350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1811302350-dwd---bin.gz
Download

Download of  raa01-sf_10000-1901282350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1901292350-dwd---bin.gz
Download of  raa01-sf_10000-1901292350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1901302350-dwd---bin.gz
Download of  raa01-sf_10000-1901302350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1901312350-dwd---bin.gz
Download of  raa01-sf_10000-1901312350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1902012350-dwd---bin.gz
Download of  raa01-sf_10000-1902012350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1902022350-dwd---bin.gz
Download of  raa01-sf_10000-1902022350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1902032350-dwd---bin.gz
Download of  raa01-sf_10000-1902032350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1902042350-dwd---bin.gz
Download of  raa01-sf_10000-1902042350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1902052350-dwd---bin.gz
Download

Download of  raa01-sf_10000-1904052350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1904062350-dwd---bin.gz
Download of  raa01-sf_10000-1904062350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1904072350-dwd---bin.gz
Download of  raa01-sf_10000-1904072350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1904082350-dwd---bin.gz
Download of  raa01-sf_10000-1904082350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1904092350-dwd---bin.gz
Download of  raa01-sf_10000-1904092350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1904102350-dwd---bin.gz
Download of  raa01-sf_10000-1904102350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1904112350-dwd---bin.gz
Download of  raa01-sf_10000-1904112350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1904122350-dwd---bin.gz
Download of  raa01-sf_10000-1904122350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1904132350-dwd---bin.gz
Download

Download of  raa01-sf_10000-1906112350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1906122350-dwd---bin.gz
Download of  raa01-sf_10000-1906122350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1906132350-dwd---bin.gz
Download of  raa01-sf_10000-1906132350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1906142350-dwd---bin.gz
Download of  raa01-sf_10000-1906142350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1906152350-dwd---bin.gz
Download of  raa01-sf_10000-1906152350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1906162350-dwd---bin.gz
Download of  raa01-sf_10000-1906162350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1906172350-dwd---bin.gz
Download of  raa01-sf_10000-1906172350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1906182350-dwd---bin.gz
Download of  raa01-sf_10000-1906182350-dwd---bin.gz succesfully!
Start downloading:  raa01-sf_10000-1906192350-dwd---bin.gz
Download

'221 Goodbye.'